In [21]:
from ultralytics import YOLO
import numpy as np
import cv2
import math

cap = cv2.VideoCapture('car.mp4')
ret, frame = cap.read()
if not ret:
    exit()

prev_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

speed = 0
feature_params = dict(maxCorners=10, qualityLevel=0.5, minDistance=7, blockSize=7)
lk_params = dict(winSize=(15, 15), maxLevel=2,
                 criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
prevPts = cv2.goodFeaturesToTrack(prev_gray, mask=None, **feature_params)
mask = np.zeros_like(frame)
fps = cap.get(cv2.CAP_PROP_FPS)
frame_time = 1 / fps
font = cv2.FONT_HERSHEY_SIMPLEX
scale_percent = 20

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    nextPts, status, err = cv2.calcOpticalFlowPyrLK(prev_gray, gray, prevPts, None, **lk_params)

    if nextPts is not None:
        good_new = nextPts[status == 1]
        good_old = prevPts[status == 1]

        for new, old in zip(good_new, good_old):
            x_new, y_new = new.ravel()
            x_old, y_old = old.ravel()
            distance = math.sqrt((x_new - x_old) ** 2 + (y_new - y_old) ** 2)
            speed = distance / frame_time
            mask = cv2.line(mask, (int(x_new), int(y_new)), (int(x_old), int(y_old)), (0, 255, 0), 2)
            frame = cv2.circle(frame, (int(x_new), int(y_new)), 5, (0, 0, 255), -1)

    frame = cv2.add(frame, mask)
    cv2.putText(frame, f'speed: {speed:.2f} px/s', (10, 80), font, 1, (0, 255, 255), 2)

    width = int(frame.shape[1] * scale_percent / 100)
    height = int(frame.shape[0] * scale_percent / 100)
    dim = (width, height)
    resized_frame = cv2.resize(frame, dim, interpolation=cv2.INTER_AREA)
    cv2.imshow("Tracker", resized_frame)

    key = cv2.waitKey(20) & 0xFF
    if key == 27:
        break

    prev_gray = gray.copy()
    prevPts = good_new.reshape(-1, 1, 2)

cap.release()
cv2.destroyAllWindows()
